<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>

***

# 2.0 ROS Integration of Image Recognition (continued)

[2.1 What is Image Recognition?](0200_DLforRoboticsImageRecognition.ipynb#image_recognition)<br>
[2.2 Image Recognition Models](0200_DLforRoboticsImageRecognition.ipynb#models)<br>
[2.3 Inference on the Robot's Jetson](#inference)<br>
[2.4 Integrating Inference into ROS](#integration)<br>
[2.5 Your ROS Image Recognition node: bot-nav classifier](#simulation)<br>
[2.6 Assessment](#assessment)<br>
[2.7 Deployment](#deployment)<br>
[2.8 Recap](#recap)<br>
[Appendix](9900_DL4R_Appendix.ipynb#apx-2)

Congratulations!  You've successfully trained a classification deep learning model for image recognition!   This is the process you can follow if you have a dataset you want to train for your project.  For our robot project, we are have a trained model already loaded.  We'll use a GoogleNet model that is pre-trained on [1000 classes of objects](https://github.com/dusty-nv/jetson-inference/blob/master/data/networks/ilsvrc12_synset_words.txt) from the ImageNet ILSVRC12 benchmark. The model is more complex and takes longer to train, but the principle is the same.

<a id='inference'></a>
# 2.3 Inference on the Robot's Jetson

In order to run inference on our robot, we need:

1. Inference software to run the neural network models on the NVIDIA Jetson TX2 GPU
2. A trained model downloaded from DIGITS
3. Software to translate the inference results into a ROS topic we can work with

The inference software and pre-trained models have already been deployed to the robot's Jetson (steps 1 and 2). The inference software is available from the the [jetson-inference](https://github.com/dusty-nv/jetson-inference#downloading-model-snapshot-to-jetson) repository, which includes detailed instructions and guides for running inference on a Jetson. Basic instructions are also provided in the [Appendix](9900_DL4R_Appendix.ipynb#apx-2). 

### About the `jetson-inference` library
The [jetson-inference](https://github.com/dusty-nv/jetson-inference#downloading-model-snapshot-to-jetson) repository uses [NVIDIA TensorRT](https://developer.nvidia.com/tensorrt) for efficiently deploying neural networks onto the embedded platform, improving performance and power efficiency using graph optimizations, kernel fusion, and half-precision FP16 on the Jetson.

![](https://raw.githubusercontent.com/dusty-nv/jetson-inference/master/docs/images/deep-vision-primitives.png)

Vision primitives, such as `imageNet` for image recognition, `detectNet` for object localization, and `segNet` for semantic segmentation, inherit from the shared `tensorNet` object. Examples are provided for streaming from live camera feed and processing images from disk. See the [Deep Vision API Reference Specification](https://rawgit.com/dusty-nv/jetson-inference/master/docs/html/index.html) for accompanying documentation.

<a id='integration'></a>
# 2.4 Integrating Inference into ROS

### `imageNet` vision primitive
Image recognition inference on the Jetson is built around the the Deep Vision [imageNet object](https://rawgit.com/dusty-nv/jetson-inference/master/docs/html/classimageNet.html).  The supporting software is installed from the `jetson-inference` library on the robot's Jetson. This makes it possible to create a neural network on the Jetson with a few programmatic commands in C++ and start the classifier on an image, or series of images in the case of a video stream.  The neural network outputs a number corresponding to which image identification is most likely, along with a confidence level.

Integration with ROS entails "wrapping" the calls in a ROS node.  This capability is included in the [ros_deep_learning](https://github.com/dusty-nv/ros_deep_learning) repository.

### About the `ros_deep_learning` package

This repo contains deep learning inference nodes for ROS with support for NVIDIA Jetson TX1/TX2/Xavier and TensorRT.

The nodes use the image recognition and object detection vision objects from the `jetson-inference` library and [NVIDIA Two Days to a Demo](https://developer.nvidia.com/embedded/twodaystoademo) tutorial, which come with several built-in pretrained network models and the ability to load customized user-trained models. ROS Kinetic (for TX1/TX2) and ROS Melodic (for Xavier) are supported.

Additional ROS package dependencies installed on the robot are:
* [image-transport](http://wiki.ros.org/image_transport)
* [image-publisher](http://wiki.ros.org/image_publisher)
* [vision-msgs](http://wiki.ros.org/vision_msgs)


### The classification message
The `image-publisher` node publishes the images from the camera on the `/image_raw` topic.  The `imagenet` node in the `ros_deep_learning` package initializes and reads the the neural network inference, and then publishes the results on the `/imagenet/classification` topic in the [vision_msgs/Classification2D](http://docs.ros.org/melodic/api/vision_msgs/html/msg/Classification2D.html) format.  To see what this format looks like, use the [rosmsg show](http://wiki.ros.org/rosmsg) command in a desktop terminal.

<img src="images/visionmsg.png" width=500>

When the message is "called back" in the node's subscriber, the elements we are interested in can be referenced.  For example, suppose your classification node is subscribing to this topic and you want to know what the result was. That information is in the ObjectHypothesis array. You could extract that information like so:

```python
my_result = msg.results[0]
rospy.loginfo("Classify callback ID: %d score: %f", my_result.id, my_result.score)
```

The rest of the message isn't needed.

<a id='simulation'></a>
# 2.5 Your ROS Image Recognition node: `bot-nav classifier`

<img src="images/nodes-classifier.png" width=600>

The bot-nav classifier node subscribes to the `/imagenet/classification` topic.  The idea is that it could also publish a movement of some sort based on what image it sees.  For example if it sees a bear, it might quickly turn around and drive away!  This could be done by publishing the appropriate movement to the `/cmd_vel` topic, just as you did in section 1.  For now though, we just want to be sure that the node is properly analyzing the classification message.

To open your `classifier` Python file, run the following cell to create a symbolic link from this notebook environment to the ROS workspace.

In [ ]:
!ln -sv ~/AionR1_ws/src/bot-nav/scripts/section2/classifier classifier

Next, click the `File/Open` drop-down command from the navigation on this notebook page.  You'll see a list of all the notebooks for the course as well as the symbolic link you just created.  Click on the `classifier` link to open the file editor.

### Testing your node with ROS bags
Our Gazebo simulation does not have a photorealistic view that can be used for actual inference in our simulation.  We can still test our responses to the `/imagenet/classification` topic input stream with the help of [ROS bags](http://wiki.ros.org/Bags).  With this tool, we can create a recording of everything a robot senses, save it, and then play the recording back.  Some ROS bags have been recorded for you for a few different images, and these can be "played" in the ROS network so that your node can respond to the messages in real time.

In [ ]:
!ls ~/bags/classify*.bag

Each of these bags contains a recording of the `/imagenet/classification` topic while the robot is viewing an object.  For example `classify1.bag` records the inference while continuously viewing a water bottle, which is class #898 in the [list of 1000 classified objects](https://github.com/dusty-nv/jetson-inference/blob/master/data/networks/ilsvrc12_synset_words.txt) (see line #899 of the list).   The only `id` the classifier should get is this water bottle! More information on the bag recordings can be found in the `~/Classifier Info` directory.

<img src="images/classify2-898-water-bottle.jpg" width=500/>

To see the ROS bag in action, you will need to start up several nodes and topic echo processes, each with its own terminal.  The first one is the ROS master node with the `roscore` command.  We didn't need this before because it was automatically started with the Gazebo launch, but we need it now since we are running ROS without Gazebo.  Here are the processes you need:

* new terminal

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:200px; 
  ">roscore
</code>
    
* new terminal

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:600px; 
  ">rosrun bot-nav classifier /cmd_vel:=/cmd_vel_mux/input/teleop
</code>
    
* new terminal

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:400px; 
  ">rostopic echo /imagenet/classification
</code>
    
* new terminal

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:400px; 
  ">rosbag play ~/bags/classify2.bag
</code>
    
You still need one more thing.  Your `classifier` node is also subscribing to the `/remote_commad` topic.  It won't start classifying until it knows it is in "guided" mode and has received a "classify" command,  To send these commands, use the [rostopic pub](http://wiki.ros.org/rostopic#rostopic_pub) command in yet another new terminal as follows:

* new terminal

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:600px; 
  ">rostopic pub /remote_command std_msgs/String "data: 'guided'" 
</code>



<img src="images/classifier_rosbag_test.png"/>

The bot-nav interface can also be used to launch rosbags.  These are set up under the Section 2 tab:

<code  style="
  border:2px solid gray;
  background-color:black;
  color:white;
  display: block;
  padding:10px; 
  width:200px; 
  ">bni
</code>

<a id='assessment'></a>
# 2.6 Assessment

### Section 2 Coding: Identify what object was found on the inference classification topic

Follow the "TODO" instructions in the `classifier` Python ROS node file.  Test your code using ROS bags either directly, or using the bot-nav interface.

```python
...
    @staticmethod
    def parse_results(classify_data):
        # TODO complete the code to build the loginfo printout of the classifier result
        # ********your code starts here - do not modify code above here
        # extract the results (ID and score) from the message
        # note that you only need to look at the first classification
        # print out the results with the loginfo command
        # return the results instead of "None"

        return None  # FIXME
        # ********your code ends here - do not modify code below here
...
```

Once your code is working correctly on the desktop, you can submit it for course assessment.  
First, copy your `classifier` node to the assessment folder by executing the following copy command. You'll need to do this any time you change your code and want to resumbit for grading.   You can do this as many times as you wish.

In [ ]:
!cp -v ~/AionR1_ws/src/bot-nav/scripts/section2/classifier assessment_export_dir

Next, return to the classroom launch page, and click the the checkmark titled "ASSESS TASK" to run the grading assessment. 

<img src="images/assess_task.png" width=800/>

<a id='deployment'></a>
# 2.7 Deployment

We won't deploy the `classification` node.  Why is that?  The imagenet model can recognize 1000 objects!  Unfortunately, it must classify the whole image within its camera view as an object.  We need to know _where the object is located,_ in order to move toward it.

You have all the building blocks now though to do just that!  All we need is a more complex inference model, which we'll cover in Section 3.

<a id='recap'></a>
# 2.8 Recap

Congratulations!  You have successfully written a classification node to recognize images inferred from a DNN model, and _make that information accessible_ to the robot logic.  Specifically, you've learned to:
* Train an Image Recognition (classification) model with DIGITS
* Access the imagenet inference stream through ROS
* Analyze the inference stream to determine what the robot has inferred from the scene
* Use ROS bags to test your code

Up next, you'll take this a step further with a more complex inference model and build a node that can recognize an object and move the robot in response! 

## [--> Next: 3.0 ROS Integration of Object Detection](0300_DLforRoboticsObjectDetection.ipynb)

<div style="border:2px solid black; background-color:#f2f2f2; padding:10px; width:400px; margin:auto;">
<b><a href="DLI Welcome Notebook.ipynb">Overview</a></b><br>
<b><a href="0100_DLforRoboticsControl.ipynb">1.0 Introduction to ROS Control</a></b><br>
<b><a href="0200_DLforRoboticsImageRecognition.ipynb">2.0 ROS Integration of Image Recognition</a></b><br>
<b><a href="0300_DLforRoboticsObjectDetection.ipynb">3.0 ROS Integration of Object Detection</a></b><br>
<b><a href="9900_DL4R_Appendix.ipynb">Appendix</a></b><br>
</div>

<a href="https://www.nvidia.com/dli"> <img src="images/DLI Header.png" alt="Header" style="width: 400px;"/> </a>